This notebook examines the effect of varying the hyperparameter $t$ on the mean task loss and financial CVaR tail risk on the battery storage problem, for the pre-trained MLP models.

In [ ]:
%load_ext autoreload
%autoreload 2

%cd ../

In [ ]:
from collections.abc import Iterable, Mapping
import itertools
import os

import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch import Tensor
from tqdm.auto import tqdm

from models.mlp import MLP
from storage.data import get_tensors, get_train_calib_split
from storage.problems import (
    StorageConstants, StorageProblemNonRobust, StorageProblemLambda)
from run_storage import cvar, get_zs

Device = str | torch.device

INPUT_DIM = 101  # including future_temp
Y_DIM = 24
MAX_PRETRAIN_EPOCHS = 500
MAX_FINETUNE_EPOCHS = 100
BATCH_SIZE = 400
PSEUDOCAILB_SIZE = 200
SEEDS = range(10)
LOG_PRICES = False
LABEL_NOISE = 20

STORAGE_CONSTS = [
    StorageConstants(lam=0.1, eps=.05),
]

sns.set_style('darkgrid')

out_dir = 'out/storage_mlp_shuffle/'

In [ ]:
def get_lams_dt(
    tensors_dict: Mapping[str, Tensor], model: MLP, prob: StorageProblemNonRobust,
    device: Device, alphas: Iterable[float], deltas: Iterable[float],
    dts: Iterable[float] = [-5, -2, -1, 0, 1, 2, 5], dt_relative: bool = False
) -> pd.DataFrame:
    """
    Returns a DataFrame with columns:
        alpha, delta, lambda, t, dt, task loss (on calib split), CVaR (on calib split)
    Note that `t` includes the `dt` offset.

    Args:
        tensors_dict: dict with keys 'X_train', 'X_calib', 'Y_train', 'Y_calib'
        model: trained MLP model
        prob: problem instance
        device: 'cpu', 'cuda', or 'cuda:X' where X is the GPU index
        alphas: risk bounds
        deltas: quantile levels
        dts: offsets to apply to `t` tuned on training set
        dt_relative: whether `dt` is a relative or absolute adjustment to the optimal `t`
            found on the training set. Relative means `t_new = (1 + dt) * t`.
            Absolute means `t_new = t + dt`.
    """
    X_train = tensors_dict['X_train'].to(device, non_blocking=True)
    X_val = tensors_dict['X_calib'].to(device, non_blocking=True)
    Y_train_np = tensors_dict['Y_train'].cpu().numpy()
    Y_val_np = tensors_dict['Y_calib'].cpu().numpy()

    # get decision variables
    with torch.no_grad():
        model.eval().to(device)
        pred_np_train = model(X_train).cpu().numpy()
        pred_np_val = model(X_val).cpu().numpy()
    z_in_train, z_out_train, z_net_train = get_zs(prob, pred_np_train)
    z_in_val, z_out_val, z_net_val = get_zs(prob, pred_np_val)

    max_lambda_prob_var_t = StorageProblemLambda(
        T=Y_DIM, const=prob.const,
        y=Y_train_np, y_mean=prob.y_mean, y_std=prob.y_std,
        z_in=z_in_train, z_out=z_out_train, z_net=z_net_train, quad=False, t_fixed=False)

    max_lambda_prob_fixed_t = StorageProblemLambda(
        T=Y_DIM, const=prob.const,
        y=Y_val_np, y_mean=prob.y_mean, y_std=prob.y_std,
        z_in=z_in_val, z_out=z_out_val, z_net=z_net_val, quad=False, t_fixed=True)

    rows = []
    for alpha, delta in itertools.product(alphas, deltas):
        max_lambda_prob_var_t.solve(alpha, delta)
        assert max_lambda_prob_var_t.t.value is not None
        t = max_lambda_prob_var_t.t.value.item()

        for dt in dts:
            if dt_relative:
                t_new = (1 + dt) * t
            else:
                t_new = t + dt

            if t_new < 0:
                rows.append({
                    'alpha': alpha, 'delta': delta, 'dt': dt,
                    'lambda': np.nan, 't': t_new,
                    'task loss': np.nan, 'CVaR': np.nan
                })
                continue

            λ = max_lambda_prob_fixed_t.solve(alpha, delta, t=t_new)
            if λ == 0.:
                val_task_loss = 0.
                val_cvar = 0.
            else:
                task_losses = prob.task_loss(
                    z_in_val * λ, z_out_val * λ, z_net_val * λ,
                    y=Y_val_np, is_standardized=True)
                financial_losses = prob.financial_loss(
                    z_in_val * λ, z_out_val * λ, y=Y_val_np, is_standardized=True)
                assert isinstance(task_losses, np.ndarray)
                assert isinstance(financial_losses, np.ndarray)
                val_task_loss = np.mean(task_losses).item()
                val_cvar = cvar(financial_losses, q=delta)

            rows.append({
                'alpha': alpha, 'delta': delta, 'dt': dt,
                'lambda': λ,
                't': t_new,
                'task loss': val_task_loss,
                'CVaR': val_cvar
            })

    return pd.DataFrame(rows).set_index(['alpha', 'delta', 'dt'])


def crc_dt(
    shuffle: bool, future_temp: bool, label_noise: float, const: StorageConstants,
    alphas: Iterable[float], deltas: Iterable[float],
    dts: Iterable[float], dt_relative: bool,
    seed: int, saved_ckpt_fmt: str, device: Device
) -> list[dict[str, float]]:
    """
    Post-hoc CRC. Always call this function within a torch.no_grad() context.

    Args:
        dts: list of offsets to apply to `t` tuned on training set
        dt_relative: whether `dt` is a relative (True) or absolute (False) adjustment to
            the optimal `t`

    Returns:
        list of dicts, one per (alpha, delta) pair, with keys:
            seed, alpha, delta, lambda, task loss, cvar
    """
    tensors, y_info = get_tensors(
        shuffle=shuffle, log_prices=LOG_PRICES, future_temp=future_temp,
        label_noise=label_noise)
    assert isinstance(y_info, tuple)
    y_mean, y_std = y_info
    tensors_cv, _ = get_train_calib_split(tensors, seed=seed)

    prob = StorageProblemNonRobust(T=Y_DIM, y_mean=y_mean, y_std=y_std, const=const)

    # load the model
    model = MLP(input_dim=tensors['X_test'].shape[1], y_dim=Y_DIM)
    saved_ckpt_path = saved_ckpt_fmt.format(seed=seed)
    model.load_state_dict(torch.load(saved_ckpt_path, weights_only=True))
    model.eval().to(device)

    calib_df = get_lams_dt(
        tensors_dict=tensors_cv, model=model,
        prob=prob, device=device, alphas=alphas, deltas=deltas,
        dts=dts, dt_relative=dt_relative)

    # use lambdas on test set
    with torch.no_grad():
        model.eval().to(device)
        pred_np = model(tensors['X_test'].to(device)).cpu().numpy()  # type: ignore
    z_in, z_out, z_net = get_zs(prob, preds=pred_np)
    y_test = tensors['Y_test'].cpu().numpy()  # type: ignore
    rows = []
    for (alpha, delta, dt) in calib_df.index:
        t = calib_df.loc[(alpha, delta, dt), 't']
        λ = calib_df.loc[(alpha, delta, dt), 'lambda']
        if np.isnan(λ):
            # if λ is NaN, we cannot compute task loss or CVaR
            rows.append({
                'seed': seed, 'alpha': alpha, 'delta': delta, 'dt': dt, 't': t,
                'lambda': np.nan,
                'task loss': np.nan,
                'task losses': np.nan,
                'cvar': np.nan
            })
            continue

        task_losses = prob.task_loss(z_in * λ, z_out * λ, z_net * λ, y=y_test, is_standardized=True)
        financial_losses = prob.financial_loss(z_in * λ, z_out * λ, y=y_test, is_standardized=True)
        assert isinstance(task_losses, np.ndarray)
        assert isinstance(financial_losses, np.ndarray)
        rows.append({
            'seed': seed, 'alpha': alpha, 'delta': delta, 'dt': dt, 't': t, 'lambda': λ,
            'task loss': np.mean(task_losses).item(),
            'task losses': task_losses,
            'financial losses': financial_losses,
            'cvar': cvar(financial_losses, q=delta)
        })

    return rows


def mean_std(row: pd.Series, mean_fmt: str = '{:.2g}', std_fmt: str = '{:.2g}') -> str:
    mean = row['mean']
    std = row['std']
    if np.isnan(mean):
        assert np.isnan(std)
        return "nan"
    return f'{mean_fmt.format(mean)} ± {std_fmt.format(std)}'

## Absolute $\Delta t$

In [ ]:
all_rows: list[dict[str, float]] = []
saved_ckpt_fmt = os.path.join(out_dir, 'mlp_s{seed}.pt')
for s in tqdm(SEEDS):
    crc_results = crc_dt(
        seed=s, shuffle=True, future_temp=False,
        label_noise=LABEL_NOISE, const=STORAGE_CONSTS[0],
        alphas=[2, 5, 10], deltas=[.9, .95, .99],
        dts=[-5, -2, -1, 0, 1, 2, 5], dt_relative=False,
        saved_ckpt_fmt=saved_ckpt_fmt, device='cuda:1')
    all_rows.extend(crc_results)
# save results to file
df = pd.DataFrame(all_rows)

In [ ]:
del df['task losses']
del df['financial losses']
df.to_csv(os.path.join(out_dir, 'crc_vary_t_absolute.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(out_dir, 'crc_vary_t_absolute.csv'))
df = df.rename(columns={'delta': 'δ', 'alpha': 'α', 'dt': 'Δt', 'lambda': 'λ'})
df = df.set_index(['α', 'δ', 'Δt', 'seed'])
df

In [ ]:
summary = (
    df[['t', 'task loss', 'cvar']]
    .groupby(['α', 'δ', 'Δt'])
    .agg(['mean', 'std'])
)

with pd.option_context('display.max_rows', 100):
    # display(pd.DataFrame({
    #     'task loss': summary['task loss'].apply(mean_std, axis=1),
    #     'cvar': summary['cvar'].apply(mean_std, axis=1)
    # }))

    fmt = '{:.1f}'
    kwargs = {'mean_fmt': fmt, 'std_fmt': fmt}

    print('t')
    print(
        summary['t'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )
    print()

    print('Task loss:')
    print(
        summary['task loss'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )
    print()
    print('CVaR:')
    print(
        summary['cvar'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )

In [ ]:
df_long = df[['task loss', 'cvar']]
df_long.columns.name = 'metric'
df_long = df_long.stack().to_frame(name='value')
df_long

In [ ]:
g = sns.catplot(
    data=df_long, x='Δt', y='value', hue='α',
    col='δ', row='metric',
    sharey=False, kind='box', height=2.8, aspect=1.4,
    width=0.5
)

# Iterate through the axes and add vertical lines
for ax in g.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)

In [ ]:
# import seaborn.objects as so

# (
#     so.Plot(df_long, x='Δt', y='value', color='α')
#     .add(so.Bar(), so.Agg(), so.Dodge())
#     .facet(col='δ', row='metric')
# )

# Relative $\Delta t$

In [ ]:
all_rows: list[dict[str, float]] = []
saved_ckpt_fmt = os.path.join(out_dir, 'mlp_s{seed}.pt')
for s in tqdm(SEEDS):
    crc_results = crc_dt(
        seed=s, shuffle=True, future_temp=False,
        label_noise=LABEL_NOISE, const=STORAGE_CONSTS[0],
        alphas=[2, 5, 10], deltas=[.9, .95, .99],
        dts=[-0.5, -0.2, -0.1, 0, 0.1, 0.2, 0.5], dt_relative=True,
        saved_ckpt_fmt=saved_ckpt_fmt, device='cuda:1')
    all_rows.extend(crc_results)
# save results to file
df = pd.DataFrame(all_rows)

In [ ]:
del df['task losses']
del df['financial losses']
df.to_csv(os.path.join(out_dir, 'crc_vary_t_relative.csv'), index=False)

In [ ]:
df = pd.read_csv(os.path.join(out_dir, 'crc_vary_t_relative.csv'))
df = df.rename(columns={'delta': 'δ', 'alpha': 'α', 'dt': 'Δt', 'lambda': 'λ'})
df = df.set_index(['α', 'δ', 'Δt', 'seed'])
df

In [ ]:
summary = (
    df[['t', 'task loss', 'cvar']]
    .groupby(['α', 'δ', 'Δt'])
    .agg(['mean', 'std'])
)

with pd.option_context('display.max_rows', 100):
    # display(pd.DataFrame({
    #     'task loss': summary['task loss'].apply(mean_std, axis=1),
    #     'cvar': summary['cvar'].apply(mean_std, axis=1)
    # }))

    fmt = '{:.1f}'
    kwargs = {'mean_fmt': fmt, 'std_fmt': fmt}

    print('t')
    print(
        summary['t'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )
    print()

    print('Task loss:')
    print(
        summary['task loss'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )
    print()
    print('CVaR:')
    print(
        summary['cvar'].apply(mean_std, axis=1, **kwargs).unstack(['α', 'δ']).to_latex()
    )

In [ ]:
df_long = df[['task loss', 'cvar']]
df_long.columns.name = 'metric'
df_long = df_long.stack().to_frame(name='value')
df_long

In [ ]:
g = sns.catplot(
    data=df_long, x='Δt', y='value', hue='α',
    col='δ', row='metric',
    sharey=False, kind='box', height=2.8, aspect=1.4,
    width=0.5
)

# Iterate through the axes and add vertical lines
for ax in g.axes.flatten():
    ticks = ax.get_xticks()
    for i in range(1, len(ticks)):
        ax.axvline((ticks[i] + ticks[i-1])/2, color="gray", linestyle="--", linewidth=1)